In [2]:
import pandas as pd

from sqlalchemy import create_engine

import pymysql
pymysql.install_as_MySQLdb()

# Change the following variables to match the season and week you need

In [3]:
season = 2018
week = 16

# Extraction of Data

In [4]:
#Extracting ESPN Projections


position_ids = [0,2,4,6] #qb=0 rb=2 wr=4 te=6
pages = [0,40]

dummy = pd.DataFrame()
for position in position_ids:
    for page in pages:
        url = f"http://games.espn.com/ffl/tools/projections?&slotCategoryId={position}&scoringPeriodId={week}&seasonId={season}&startIndex={page}"
        df = pd.read_html(url,header=1)
        df = df[1]
        if position == 0:
            pos = "QB"
        if position == 2:
            pos = "RB"
        if position == 4:
            pos = "WR"
        if position == 6:
            pos = "TE"
        df = df['PLAYER, TEAM POS'].str.split(',',expand=True)
        df['Position'] = pos
        if position == 0 and page == 0:
            main = pd.concat([dummy,df])
        else:
            main = pd.concat([main,df])
main.to_csv(f'resources/espn_projections/espn-week{week}-projections.csv')

C:\Users\frank\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [5]:
#Extracting Site 2 
csv_file = f'Resources/Site_2_Projections/Site_2_W{week}_projections.csv'
four44_df = pd.read_csv(csv_file)

In [6]:
#Extracting Fantasy Sharks
csv_file = f'Resources/Fantasy_Sharks_Projections/W{week}_Projections.csv'

#Resources\Fantasy_Sharks_Projections\fantasysharks_W16_Projections.csv
sharks_df = pd.read_csv(csv_file)

,Rank,Pos,ID,Name,Team,Opp,Comp,Yards,TD,Int,Att,Yards.1,TD.1,Rec,Yards.2,TD.2,Fantasy Points
0,1,QB,13116,"Mahomes, Patrick",KCC,SEA,27.39,349,3.17,0.64,3.52,16,0.11,0.00,0,0.00,32
1,2,QB,10703,"Wilson, Russell",SEA,KCC,22.20,262,2.56,0.41,4.12,24,0.07,0.00,0,0.00,26
2,3,QB,9099,"Ryan, Matt",ATL,CAR,27.11,310,1.93,0.49,1.79,8,0.15,0.00,0,0.00,25
3,4,RB,13130,"McCaffrey, Christian",CAR,ATL,0.00,0,0.00,0.00,12.49,59,0.46,6.68,56,0.44,25
4,5,QB,7401,"Roethlisberger, Ben",PIT,NOS,26.70,308,2.15,0.96,1.73,5,0.12,0.00,0,0.00,25


# Transformation of Data

In [ ]:
#transforming ESPN Data

In [ ]:
#transforming Site 2

# Applying Fanduel uniform scoring system to Fantasy Sharks raw predictions and creating a new column with the results
four44_df['Site 2'] = four44_df['Pass Yds'] * .04 + four44_df['Rush Yds'] * .1 + four44_df['Rec Yds'] * .1 + \
four44_df['Pass TD'] * 4 + four44_df['Rush TD'] * 6 + four44_df['Rec TD'] * 6 + four44_df['Rec'] * .5 + four44_df['INT'] * -1 \
+ four44_df['Fum'] * -2

#dropping unneeded columns
four44_df = four44_df.drop(['PID','Pos','Team','Opp','aFPA','aFPA Rk','FFPts','Comp','Pass Att','Pass Yds','Pass TD','INT',
'Rush Att','Rush Yds','Rush TD','Rec','Rec Yds','Rec TD','Pa1D','Ru1D','Rec1D','Fum','XP','FG','Grade'], axis=1)

# Rename player column to "Name"
four44_df.rename(columns = {"Player": "Name"}, inplace = True)

In [ ]:
#transforming Fantasy Sharks

# Splitting the name into 2 seperate columns
sharks_df[['Last Name','First Name']] = sharks_df['Name'].str.split(', ',expand=True)

# Recombine names into one column in the order of First Last
sharks_df['Name'] = sharks_df['First Name'].str.cat(sharks_df['Last Name'],sep=" ")

# Applying Fanduel uniform scoring system to Fantasy Sharks raw predictions and creating a new column with the results
sharks_df['Fantasy Sharks'] = sharks_df['Yards'] * .04 + sharks_df['Yards.1'] * .1 + sharks_df['Yards.2'] * .1 + \
sharks_df['TD'] * 4 + sharks_df['TD.1'] * 6 + sharks_df['TD.2'] * 6 + sharks_df['Rec'] * .5 + sharks_df['Int'] * -1 

#Removing unneeded columns
sharks_df = sharks_df.drop(['ID', 'Rank', 'First Name', 'Last Name', 'Comp', 'Yards', 'TD', 'TD.1', 
'TD.2', 'Int', 'Att', 'Yards.1', 'Yards.2', 'Rec', 'Fantasy Points'], axis=1)


In [ ]:
#combining cleaned DataFrames into one DataFrame

combined_df = sharks_df.merge(four44_df, on=['Name'], sort=False)

combined_df.round({'Fantasy Sharks': 1, 'Site 2': 1})


pd.options.display.float_format = '{:,.2f}'.format

combined_df = combined_df[['Season', 'Week', 'Name', 'Pos', 'Team', 'Opp', 'Fantasy Sharks', 'Site 2']]

combined_df.head()